In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [2]:
df = pd.read_csv('tweets.csv',encoding='latin-1')
df

,Id,Fecha,Tweet,Usuario,Clasificación
0,3,6/1/2022,Una ultima prueba de cargar tweets,alejandro,Neutro
1,4,5/31/2022,esta es una prueba de tweet enviado,alejandro,Neutro
2,11,3/3/2022,hola profe,miusuario,Neutro
3,47,6/13/2022,La FAA ha autorizado a SpaceX a utilizar su pu...,Guido de Caso :rocket:,Neutro
4,62,6/13/2022,"Es de las conspiranoicas,de las que creen que ...",:sparkles::cherryblossom: :cherryblossom::spa...,Positivo
...,...,...,...,...,...
2058,6704,6/21/2022,#felizlunes yo pienso que es posible para las ...,lfinanciera1,Neutro
2059,6706,6/21/2022,hija transgénero de elon musk xavier musk soli...,elimparcialcom,Neutro
2060,6728,6/21/2022,twitter fue comprado por elon musk y ahora cam...,hwamystar,Neutro
2061,6731,6/21/2022,urge ver este tipo de noticias inundar la web ...,yurak,Negativo


In [3]:
df1=df.loc[:,['Tweet','Clasificación']]
df1

,Tweet,Clasificación
0,Una ultima prueba de cargar tweets,Neutro
1,esta es una prueba de tweet enviado,Neutro
2,hola profe,Neutro
3,La FAA ha autorizado a SpaceX a utilizar su pu...,Neutro
4,"Es de las conspiranoicas,de las que creen que ...",Positivo
...,...,...
2058,#felizlunes yo pienso que es posible para las ...,Neutro
2059,hija transgénero de elon musk xavier musk soli...,Neutro
2060,twitter fue comprado por elon musk y ahora cam...,Neutro
2061,urge ver este tipo de noticias inundar la web ...,Negativo


In [4]:
df1["Clasificación"].value_counts()

Neutro      1102
Positivo     487
Negativo     474
Name: Clasificación, dtype: int64

In [5]:
df2 =  df1[df1["Clasificación"]!="Neutro"]
df2

,Tweet,Clasificación
4,"Es de las conspiranoicas,de las que creen que ...",Positivo
5,si elon musk saca un chip neuronal ME LO VOY A...,Positivo
6,elon Musk t odio m debes 5 de hachís,Negativo
8,lo siento twitter era un chiste aunque nose si...,Negativo
9,"Tras lo sucedido con la IA de Google, nuevamen...",Positivo
...,...,...
2054,rt elon musk se enfrenta a una demanda de us 2...,Negativo
2055,elon regalame un tesla porfa,Positivo
2056,yaningnoviecita deskansa en paz elon musk no t...,Negativo
2061,urge ver este tipo de noticias inundar la web ...,Negativo


In [6]:
import string
punctuation = set(string.punctuation)
def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens
df2.head()["Tweet"].apply(tokenize)

4    [es, de, las, conspiranoicasde, las, que, cree...
5    [si, elon, musk, saca, un, chip, neuronal, me,...
6       [elon, musk, t, odio, m, debes, 5, de, hachís]
8    [lo, siento, twitter, era, un, chiste, aunque,...
9    [tras, lo, sucedido, con, la, ia, de, google, ...
Name: Tweet, dtype: object

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
demo_vectorizer = CountVectorizer(
    tokenizer = tokenize,
    binary = True
)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df2["Tweet"], df2["Clasificación"], stratify=df2["Clasificación"])
print(f"Training examples: {len(X_train)}, testing examples {len(X_test)}")

Training examples: 720, testing examples 241


In [9]:
X_train

1230    le deseo muchos exitos en la empresa por conse...
1963    demandan a elon musk con 258 000 mdd por esque...
1103    Elon musk significa "hijo de la luz"\n\nPara t...
1492     Este debe estar en alguno de los satélites qu...
182     una fila de luces se vera en el cielo y elon m...
                              ...                        
896     #BMW PROBARÁ LA BATERIA AVANZADA DE ONE EN SU ...
648     Te amo Elon musk  heart_on_fire  https://t.co/...
129     Dogecoin ha caído más del 90 desde que Elon Mu...
1747    ElonMusk no sabe perder Aplica la de noigo noi...
1743    ¡O trabajas en la oficina o buscas trabajo! Es...
Name: Tweet, Length: 720, dtype: object

In [10]:
X_test

1155    #FraseDelDía :El mayor fallo que he cometido (...
1090               viste boluda ya dije elon musk me odia
1532    Cuando el filtro que te pone triste cuando son...
1116    #Tesla recortará parte de su plantilla de trab...
783      Exijo que Elon Musk ponga el "Me divierte:, e...
                              ...                        
938       Osea que si tú tienes los padres de Jeff Bez...
1621    :DE NIÑO NERD DE SUDÁFRICA A SER EL HOMBRE MAS...
1123    iba a usar una block chain en la cuenta de elo...
1898                                   elonmusk hello sir
1775    SpaceNosey SpaceX elonmusk SpaceX debe separar...
Name: Tweet, Length: 241, dtype: object

In [11]:
real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
train_X = real_vectorizer.fit_transform(X_train)
test_X = real_vectorizer.transform(X_test)

In [12]:
from sklearn.svm import LinearSVC
classifier = LinearSVC()
classifier.fit(train_X, y_train)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

LinearSVC()

In [13]:
from sklearn.metrics import accuracy_score
predicciones = classifier.predict(test_X)
accuracy = accuracy_score(y_test, predicciones)
print(f"Accuracy: {accuracy:.4%}")

Accuracy: 68.8797%


In [14]:
#VotingClassifier
#Regresión Logística + Ridge
ridge=linear_model.LogisticRegression(max_iter= 20000,penalty='l2',fit_intercept=True, random_state=123)
#kNN
knn=KNeighborsClassifier()
#Árbol
tree=DecisionTreeClassifier(random_state=123)
#SMV
svm=SVC( random_state=123, probability=True)


In [15]:
modelos_lista=(ridge, knn, tree, svm)

for modelo in modelos_lista:
    modelo.fit(train_X,y_train)
    AC=modelo.score(test_X,y_test)
    print("AC: ",AC)

AC:  0.6556016597510373
AC:  0.5726141078838174
AC:  0.6721991701244814
AC:  0.6348547717842323


In [16]:
from sklearn.ensemble import VotingClassifier
modelos=[('ridge',ridge),('knn',knn),('arbol',tree),('svm',svm)]
votacion=VotingClassifier(estimators=modelos,voting='soft',n_jobs=-1, weights=[2,1,3,1])
votacion.fit(train_X, y_train)
AC_votacion=votacion.score(test_X,y_test)
print("AC_votacion: ",AC_votacion)

AC_votacion:  0.6763485477178424


In [17]:
#Creamos el modelo Extremely Randomized Trees
from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier(n_estimators=300,n_jobs=-1,random_state=123)
etc.fit(train_X, y_train)
etc.score(test_X,y_test)

0.6597510373443983

In [18]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV

etc1=ExtraTreesClassifier(n_jobs=-1,random_state=123)
grid_etc1= GridSearchCV(estimator=etc1, param_grid={'n_estimators':[100,200,400,800,1600]}, cv=5,n_jobs=-1, return_train_score=True)

grid_etc1.fit(train_X,y_train)

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(n_jobs=-1, random_state=123),
             n_jobs=-1, param_grid={'n_estimators': [100, 200, 400, 800, 1600]},
             return_train_score=True)

In [19]:
ACGrid=grid_etc1.best_score_
ACGrid

0.6791666666666667

In [20]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=300,n_jobs=-1,random_state=123)
rf.fit(train_X, y_train)
rf.score(test_X,y_test)

0.6763485477178424

In [21]:
rf1=RandomForestClassifier(n_jobs=-1,random_state=123)
grid_rf1= GridSearchCV(estimator=rf1, param_grid={'n_estimators':[200,400,600,800,1000]}, 
                       cv=5,n_jobs=-1, return_train_score=True)
grid_rf1.fit(train_X,y_train)
ACrf=grid_rf1.best_score_
ACrf
grid_rf1.best_params_

{'n_estimators': 1000}

In [22]:
ACrf

0.6666666666666667

In [23]:
df3 = pd.read_csv('sin clasificar.csv',encoding='latin-1')
df3 = df.loc[:,['Tweet','Clasificación']]
df3

,Tweet,Clasificación
0,Una ultima prueba de cargar tweets,Neutro
1,esta es una prueba de tweet enviado,Neutro
2,hola profe,Neutro
3,La FAA ha autorizado a SpaceX a utilizar su pu...,Neutro
4,"Es de las conspiranoicas,de las que creen que ...",Positivo
...,...,...
2058,#felizlunes yo pienso que es posible para las ...,Neutro
2059,hija transgénero de elon musk xavier musk soli...,Neutro
2060,twitter fue comprado por elon musk y ahora cam...,Neutro
2061,urge ver este tipo de noticias inundar la web ...,Negativo


In [24]:

import string
punctuation = set(string.punctuation)
def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens
df3.head()["Tweet"].apply(tokenize)

0            [una, ultima, prueba, de, cargar, tweets]
1          [esta, es, una, prueba, de, tweet, enviado]
2                                        [hola, profe]
3    [la, faa, ha, autorizado, a, spacex, a, utiliz...
4    [es, de, las, conspiranoicasde, las, que, cree...
Name: Tweet, dtype: object

In [ ]:
predicciones = classifier.predict(test_X)
accuracy = accuracy_score(y_test, predicciones)
print(f"Accuracy: {accuracy:.4%}")